In [109]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from math import sqrt
pd.set_option('display.max_columns', 100)

In [85]:
df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')
df_riders = pd.read_csv('Riders.csv')
#df_train.info()

## Time 

In [86]:
train_time_col = ['Placement', 'Confirmation', 'Arrival at Pickup', 'Pickup', 'Arrival at Destination']
test_time_col  = ['Placement', 'Confirmation', 'Arrival at Pickup', 'Pickup']
time_cat = ['Day of Month', 'Weekday (Mo = 1) ', 'Time']
cat_col = ['Personal or Business']
drop_columns_train = ['User Id','Vehicle Type', 'Temperature', 'Precipitation in millimeters', 
                        'Pickup Lat','Pickup Long',
                        'Destination Lat', 'Destination Long',
                        'Arrival at Destination - Day of Month',
                        'Arrival at Destination - Weekday (Mo = 1)',
                        'Arrival at Destination - Time', 'Rider Id' ]
drop_columns_test = ['User Id','Vehicle Type', 'Temperature', 'Precipitation in millimeters', 
                        'Pickup Lat','Pickup Long',
                        'Destination Lat', 'Destination Long', 'Rider Id' ]



In [87]:
for col in test_time_col:
    time_col = col + ' - Time'
    df_train[time_col] = pd.to_datetime(df_train[time_col])
    df_test[time_col] = pd.to_datetime(df_test[time_col])    
    df_train[time_col] = df_train[time_col].dt.hour * 60 + df_train[time_col].dt.minute
    df_test[time_col] = df_test[time_col].dt.hour * 60 + df_test[time_col].dt.minute    

In [88]:
df_train = df_train.drop(drop_columns_train, axis=1)
df_test = df_test.drop(drop_columns_test, axis=1)

## Categorical

In [89]:
df_train['Personal or Business'] = df_train['Personal or Business'].astype('category').cat.codes
df_test['Personal or Business'] = df_test['Personal or Business'].astype('category').cat.codes

In [92]:
df_train.set_index('Order No', inplace=True)
df_test.set_index('Order No', inplace=True)

## Test train split 

In [ ]:
lable = df_train.pop('Time from Pickup to Arrival')

In [98]:
X_train, X_test, y_train, y_test = train_test_split(df_train, lable, test_size=0.3, random_state=42)

In [106]:
model = XGBRegressor()
model.fit(X_train, y_train)

/anaconda3/envs/py36_ml/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [110]:
sqrt(mean_squared_error(y_test, model.predict(X_test)))

794.2759557820622

## Submit

In [118]:
result = model.predict(df_test)
ids = df_test.index
submit = pd.DataFrame({'Order_No':ids,'Time from Pickup to Arrival':result.astype(int)}).set_index('Order_No')
submit.to_csv('submit.csv')